## **Product Recommender Example**

In this example, we started with the out-of-the-box recommender model from the Skafos iOS Quickstart flow and modified it for a different use-case.

---

Execute each cell one-by-one, by selecting the cell and do one of the following:

-  Clicking the "play" button at the top of this frame.
-  Typing 'Control + Enter' or 'Shift + Enter'.

In [ ]:
# If this is your first time in the JupyterLab workspace - install external dependencies
from utilities.dependencies import install
install(timeout=500)

# No need to do this in the future for this notebook

In [ ]:
# Import necessary libraries
import os
import json
import random
from time import sleep

from tqdm import tqdm
import pandas as pd
from skafossdk import *
import turicreate as tc

import utilities.save_models as sm
from utilities.thingiverse import Thingiverse

In [ ]:
# Connect to thingiverse API
# Make sure you have env vars set for you API creds
tv = Thingiverse(
    appinfo={
        'client_id': os.environ['CLIENT_ID'],
        'client_secret': os.environ['CLIENT_SECRET']
    }
)
tv.connect(token=os.environ['APP_TOKEN'])

In [ ]:
# Grab a bunch of "Things" - need a starting point
popular = tv.get_popular_things()
featured = tv.get_featured_things()
newest = tv.get_newest_things()

things_starting_list = popular + featured + newest
thing_ids = set([thing['id'] for thing in things_starting_list])
print(f'Retrieved a list of {len(thing_ids)} things from API', flush=True)

In [ ]:
# Go out and get all the users who like these starting products
thing_likes = {}

for thing_id in tqdm(thing_ids):
    try:
        user_likes = tv.get_thing_likes(thing_id)
        if user_likes:
            user_ids = set([user['id'] for user in user_likes])
            thing_likes[thing_id] = list(user_ids)
    except:
        print('skipping')

In [ ]:
# Write data out to file just in case
#with open('thing_likes.txt', 'w') as file:
#     file.write(json.dumps(thing_likes))

In [ ]:
# Grab a small sample of users from the full group
sample = 2000
all_user_ids = set(u for vals in thing_likes.values() for u in vals)
sampled_ids = random.sample(all_user_ids, 2000)
print(f'Gathering user likes for {sample/len(all_user_ids)} percent of user ids', flush=True)

In [ ]:
# Now do a reverse search on users to get items they liked
user_likes = {}
for user_id in tqdm(sampled_ids):
    try:
        things_liked = tv.get_likes_user(user_id)
        if things_liked:
            things_liked_ids = list(set(thing['id'] for thing in things_liked))
            user_likes[user_id] = things_liked_ids
            sleep(.3)
    except:
        print('skipping')
        continue
        sleep(1)

In [ ]:
# Convert data to a dataframe
cols = ['userId', 'thingId']
user_likes_df = pd.DataFrame([(k, v) for k in user_likes for v in user_likes[k]], columns=cols)
thing_likes_df = pd.DataFrame([(k, v) for k in thing_likes for v in thing_likes[k]], columns=['thingId', 'userId'])
likes = pd.concat((user_likes_df[cols], thing_likes_df[cols]), axis=0)

In [ ]:
likes.head()

In [ ]:
# Filter down the dataframe to include users who have liked atleast 5 things
like_threshold = 5
likes = likes.groupby("userId").filter(lambda x: len(x) > 5)

### 2. **Build the model**
We use the `tc.recommender.create` function (default) and specify the data, user id, and item id to properly train the model. To understand more about this specific function, check out the [Turi Create Documentation](https://apple.github.io/turicreate/docs/userguide/recommender/).

In [ ]:
# Train the default recommender
model = tc.recommender.create(
    observation_data=tc.SFrame(likes),
    user_id='userId',
    item_id='thingId'
)

In [ ]:
# Make a recommendation to a known user 
# k ~ is the number of recommendations to generate
model.recommend(
    users=[likes['userId'][0]],
    k=5
)

In [ ]:
# Make a recommendation to a new (unknown) user - returns most popular things
model.recommend(
    users=[9999999999],
    k=5
)

### 3. **Save the model**
Once your model has been created, it must be saved to the Skafos framework via the code below. This will trigger a push to your mobile app.

In [ ]:
## Export to coreml
ska = Skafos()

# Specify the CoreML model name
coreml_model_name = 'ProductRecommender.mlmodel'
# Specify the name of the compressed model we save to Skafos
compressed_model_name = coreml_model_name + ".gz" 

# Export the trained model to CoreML format
res = model.export_coreml(coreml_model_name) 

# Compress the model
compressed_model = sm.compress_model(coreml_model_name)

# Save to Skafos
sm.skafos_save_model(
    skafos=ska,
    model_name=compressed_model_name,
    compressed_model=compressed_model,
    permissions='public'
)